# Start Langchain with Gemini  
제작 : 박광석(모두의연구소, https://www.linkedin.com/in/andkspark)   
  
#### 해당 노트는 Langchain을 처음 접하시는 분들을 위한 튜토리얼입니다.  
참고 :
https://python.langchain.com/docs/get_started/introduction  
https://python.langchain.com/docs/integrations/llms/google_ai/  
YouTube <모두의AI>  
인프런 <모두를 위한 대규모 언어 모델 LLM(Large Language Model) Part 2 - 랭체인(LangChain)으로 나만의 ChatGPT 만들기>


In [1]:
YOUR_API_KEY =

### Step 0 : 설치와 준비  
Langchain 설치 및 Gemini API 키를 등록하도록 합니다.  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -U langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [4]:
import os
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

### Step 1 : LLM 사용해보기  
굉장히 간단하게 LLM을 사용할 수 있습니다!  
llm을 정의한 후, 답변을 얻기 위해, llm.invoke()를 사용할 수 있습니다.

In [5]:
#랭체인의 구글Api를 사용합니다
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", temperature=0.0, google_api_key = YOUR_API_KEY,max_output_tokens = 200)

request = llm.invoke("파이썬에 대해 설명해줘")
print(request)


content='파이썬(Python)은 고급 프로그래밍 언어로, **읽기 쉽고 배우기 쉬운 문법**을 특징으로 합니다.  다양한 분야에서 사용되며, 초보자부터 전문가까지 폭넓게 사용하는 인기 있는 언어입니다.\n\n**주요 특징:**\n\n* **인터프리터 언어:** 컴파일 과정 없이 코드를 한 줄씩 실행합니다. 이로 인해 개발 속도가 빠르고 디버깅이 용이합니다.  하지만 컴파일 언어보다 실행 속도는 느릴 수 있습니다.\n* **동적 타이핑:** 변수의 자료형을 명시적으로 선언할 필요가 없습니다.  런타임 시에 자료형이 결정됩니다.  유연성을 제공하지만, 런타임 에러 발생 가능성도 높아집니다' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'MAX_TOKENS', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--5db5800e-2f4d-4adb-b871-1d8f7c5dd2e5-0' usage_metadata={'input_tokens': 8, 'output_tokens': 200, 'total_tokens': 208, 'input_token_details': {'cache_read': 0}}


Query의 결과에는 답변 뿐만이 아닌, 메타데이터와 실행id가 함께 리턴됩니다.  
답변을 나타내는 content 키만 출력하도록 해보겠습니다.  

In [6]:
print(request.content)

파이썬(Python)은 고급 프로그래밍 언어로, **읽기 쉽고 배우기 쉬운 문법**을 특징으로 합니다.  다양한 분야에서 사용되며, 초보자부터 전문가까지 폭넓게 사용하는 인기 있는 언어입니다.

**주요 특징:**

* **인터프리터 언어:** 컴파일 과정 없이 코드를 한 줄씩 실행합니다. 이로 인해 개발 속도가 빠르고 디버깅이 용이합니다.  하지만 컴파일 언어보다 실행 속도는 느릴 수 있습니다.
* **동적 타이핑:** 변수의 자료형을 명시적으로 선언할 필요가 없습니다.  런타임 시에 자료형이 결정됩니다.  유연성을 제공하지만, 런타임 에러 발생 가능성도 높아집니다


### Step2 : schema 사용해보기  

Schema는 agent와 원활하게 작업할 수 있도록 돕는 메세지 추상화 개념입니다.  
  
SystemMessage : LLM에 역할을 부여할 수 있습니다. 대화의 맥락을 부여하며, 행동 설정 혹은 제약을 위해 사용할 수 있습니다.  
HumanMessage : 사용자가 LLM에게 대화 혹은 요청을 위해 보내는 메세지입니다.  
AIMessage : AI assistent가 제공하는 메세지, 즉 대답입니다.  
이외에도 function call 로부터 주어진 message인 FunctionMessage가 있습니다.


In [7]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [8]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Korean."),
    HumanMessage(content="Translate this sentence: I love programming."),
]

# 현재 SystemMessages 가 지원되지 않아, convert_system_message_to_human=True 의 option을 llm 생성시에 사용해야 합니다
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", convert_system_message_to_human=True)
response = llm.invoke(messages)
print(response.content)

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


나는 프로그래밍을 좋아해요. (naneun peurogeuramming-eul joa haeyo)

This is a polite and common way to say it.  You could also say:

나는 프로그래밍이 좋아요. (naneun peurogeuramming-i joa haeyo) - This is also polite, but slightly different in nuance.  It emphasizes the *act* of programming itself being enjoyable rather than the act of *doing* programming.  The difference is subtle.


In [9]:
response

AIMessage(content='나는 프로그래밍을 좋아해요. (naneun peurogeuramming-eul joa haeyo)\n\nThis is a polite and common way to say it.  You could also say:\n\n나는 프로그래밍이 좋아요. (naneun peurogeuramming-i joa haeyo) - This is also polite, but slightly different in nuance.  It emphasizes the *act* of programming itself being enjoyable rather than the act of *doing* programming.  The difference is subtle.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--509f7590-3e97-4642-b6f7-f99b3a060b33-0', usage_metadata={'input_tokens': 19, 'output_tokens': 114, 'total_tokens': 133, 'input_token_details': {'cache_read': 0}})

AI의 답변은 AIMessage로 분류되어 있습니다.

### Step3 : Prompt Templete 사용해보기  
모델에 입력하는 prompt를 규격화하여 효율적으로 LLM을 사용할 수 있게 하는 모듈입니다.  
  
ex) "파란색 셔츠" 와 잘 어울리는 "바지"를 추천해줘!   
ex) "흰 티셔츠" 와 잘 어울리는  "모자"를 추천해줘!  
  
이렇게 예시의 반복되는 부분을 엔지니어링 할 수 있습니다.  

    
PromptTemplete : 일반적인 프롬프트 템플릿 생성 시 사용합니다.  
{ }로 묶인 부분을 변수로 활용합니다  


In [10]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{meat}를 맛있게 만드는 좋은 방법은?")
prompt.format(meat="닭고기")

'닭고기를 맛있게 만드는 좋은 방법은?'

ChatPromptTemplete : 채팅 LLM 특화, 위의 schema처럼 역할을 나누어 사용하도록 입력할 수 있습니다.  

In [11]:
from langchain.prompts import ChatPromptTemplate

templete = "당신은 최고의 번역가입니다 {input_language}를 {output_language}로 번역해서 알려주세요"
human_templete = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", templete),
    ("human", human_templete),
])

format_message = chat_prompt.format_messages(input_language="English", output_language = "Korean", text = "I love pizza")

위의 Format한 prompt를 LLM에 넣어 결과를 확인할 수 있습니다.  

In [12]:
response = llm.invoke(format_message)
print(response.content)

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


피자를 정말 좋아해요.


### Step4 : Parser 사용해보기  

Parser는 llm에서 response로 넘어가는 text를 원하는 형태로 파싱하도록 도와주는 모듈입니다  

In [13]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
  #baseOutputParser를 상속받음
  #comma를 기준으로 문자열을 나누어주는 parser를 사용해보겠습니다
    def parse(self, text: str) -> list:
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hello , Hyundai")

['hello ', 'Hyundai']

### Step5 : Chain 사용하여 결합해보기  
Chain 은 Langchain의 핵심 기능으로, 위의 과정들을 묶어 한번에 수행할 수 있도록 합니다.  
다른 langchain의 구성끼리,  | 기호를 사용하여 연결할 수 있습니다.

In [14]:
templete = """당신은 쉼표로 구분된 목록을 생성하는 유용한 조수입니다. \
사용자가 카테고리를 전달하면 해당 카테고리에 속하는 5개의 객체를 쉼표로 구분된 목록으로 생성합니다. \
오직 쉼표로 구분된 목록만 반환하고 그 이상은 반환하지 마세요."""

human_templete = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", templete),
    ("human", human_templete),
])

위에 정의한 프롬프트 탬플릿, LLM, Parser를 결합해보겠습니다.

In [15]:
chain = chat_prompt | llm | CommaSeparatedListOutputParser()
response = chain.invoke ("text : 색깔")
print(response)

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


['빨강', '파랑', '초록', '노랑', '보라']


수고하셨습니다!  